In [10]:
import pandas as pd
import numpy as np
from surprise import Dataset

In [66]:
df_title= pd.read_csv('Movie_Id_Titles')
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df_title.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   item_id  1682 non-null   int64 
 1   title    1682 non-null   object
dtypes: int64(1), object(1)
memory usage: 26.4+ KB


In [67]:
movielens = Dataset.load_builtin('ml-100k')
movielens = pd.DataFrame(movielens.raw_ratings, columns = ['user_id', 'item_id', 'rating', 'timestamps'])
movielens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     100000 non-null  object 
 1   item_id     100000 non-null  object 
 2   rating      100000 non-null  float64
 3   timestamps  100000 non-null  object 
dtypes: float64(1), object(3)
memory usage: 3.1+ MB


In [68]:
movielens[['user_id', 'item_id', 'rating']] = movielens[['user_id', 'item_id', 'rating']].astype('int64')
movielens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   user_id     100000 non-null  int64 
 1   item_id     100000 non-null  int64 
 2   rating      100000 non-null  int64 
 3   timestamps  100000 non-null  object
dtypes: int64(3), object(1)
memory usage: 3.1+ MB


In [69]:
movielens = pd.merge(movielens, df_title, on = 'item_id')

In [70]:
movielens.head()

,user_id,item_id,rating,timestamps,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [99]:
movielens.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   user_id     100000 non-null  int64 
 1   item_id     100000 non-null  int64 
 2   rating      100000 non-null  int64 
 3   timestamps  100000 non-null  object
 4   title       100000 non-null  object
dtypes: int64(3), object(2)
memory usage: 4.6+ MB


In [101]:
movielens[movielens['item_id'] == 50]

,user_id,item_id,rating,timestamps,title
50711,290,50,5,880473582,Star Wars (1977)
50712,79,50,4,891271545,Star Wars (1977)
50713,2,50,5,888552084,Star Wars (1977)
50714,8,50,5,879362124,Star Wars (1977)
50715,274,50,5,878944679,Star Wars (1977)
...,...,...,...,...,...
51289,749,50,5,878846978,Star Wars (1977)
51290,886,50,5,876031501,Star Wars (1977)
51291,739,50,4,886958895,Star Wars (1977)
51292,903,50,5,891031329,Star Wars (1977)


In [93]:
moviematrix = movielens.pivot_table(index = 'title', columns = 'user_id', values = 'rating')

In [102]:
cor = moviematrix.corr(method = 'pearson')

In [107]:
cor[290].sort_values(ascending = False).head(50)

user_id
451    1.000000
651    1.000000
131    1.000000
863    1.000000
252    1.000000
520    1.000000
290    1.000000
578    1.000000
362    1.000000
724    1.000000
750    0.944911
209    0.942809
489    0.927173
797    0.912871
462    0.912871
799    0.904534
814    0.870388
134    0.866025
444    0.866025
482    0.866025
423    0.852803
742    0.840152
3      0.818182
310    0.788342
441    0.787296
590    0.759257
598    0.755929
212    0.752549
367    0.750000
104    0.748798
359    0.745957
614    0.745356
333    0.744700
433    0.734489
583    0.730297
789    0.720577
526    0.711512
2      0.707107
564    0.699854
93     0.692935
369    0.683130
185    0.671105
257    0.659380
584    0.659047
113    0.657794
822    0.645497
735    0.645497
304    0.645497
375    0.642857
227    0.640257
Name: 290, dtype: float64